# SCRIPT DE VALIDAÇÃO COMPLETA

In [0]:
-- 1.1 Listar todas as tabelas Gold
SHOW TABLES IN gold.data_suicide;

-- 1.2 Verificar schemas
DESCRIBE TABLE gold.data_suicide.suicide_hdi_analysis;

In [0]:
DESCRIBE TABLE gold.data_suicide.gender_disparity_analysis;

In [0]:
DESCRIBE TABLE gold.data_suicide.country_rankings;

In [0]:

DESCRIBE TABLE gold.data_suicide.global_metrics;

In [0]:
DESCRIBE TABLE gold.data_suicide.significant_changes_alerts;

In [0]:
DESCRIBE TABLE gold.data_suicide.momentum_analysis;

In [0]:
SELECT 
    'suicide_hdi_analysis' AS tabela,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT country_code) AS paises,
    COUNT(DISTINCT year) AS anos,
    MIN(year) AS ano_min,
    MAX(year) AS ano_max
FROM gold.data_suicide.suicide_hdi_analysis

UNION ALL

SELECT 
    'gender_disparity_analysis' AS tabela,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT country_code) AS paises,
    COUNT(DISTINCT year) AS anos,
    MIN(year) AS ano_min,
    MAX(year) AS ano_max
FROM gold.data_suicide.gender_disparity_analysis

UNION ALL

SELECT 
    'country_rankings' AS tabela,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT country_code) AS paises,
    COUNT(DISTINCT year) AS anos,
    MIN(year) AS ano_min,
    MAX(year) AS ano_max
FROM gold.data_suicide.country_rankings

UNION ALL

SELECT 
    'global_metrics' AS tabela,
    COUNT(*) AS total_registros,
    0 AS paises,
    COUNT(DISTINCT year) AS anos,
    MIN(year) AS ano_min,
    MAX(year) AS ano_max
FROM gold.data_suicide.global_metrics

UNION ALL

SELECT 
    'significant_changes_alerts' AS tabela,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT country_code) AS paises,
    COUNT(DISTINCT year) AS anos,
    MIN(year) AS ano_min,
    MAX(year) AS ano_max
FROM gold.data_suicide.significant_changes_alerts;

In [0]:
-- 3.1 Verificar valores nulos em colunas críticas
SELECT 
    'suicide_hdi_analysis - Valores Nulos' AS verificacao,
    COUNT(*) AS total_linhas,
    SUM(CASE WHEN country_code IS NULL THEN 1 ELSE 0 END) AS country_code_nulos,
    SUM(CASE WHEN year IS NULL THEN 1 ELSE 0 END) AS year_nulos,
    SUM(CASE WHEN suicide_rate IS NULL THEN 1 ELSE 0 END) AS suicide_rate_nulos,
    SUM(CASE WHEN hdi IS NULL THEN 1 ELSE 0 END) AS hdi_nulos
FROM gold.data_suicide.suicide_hdi_analysis;

-- 3.2 Verificar ranges de valores
SELECT 
    'Validação de Ranges' AS verificacao,
    MIN(suicide_rate) AS taxa_minima,
    MAX(suicide_rate) AS taxa_maxima,
    AVG(suicide_rate) AS taxa_media,
    MIN(hdi) AS hdi_minimo,
    MAX(hdi) AS hdi_maximo,
    AVG(hdi) AS hdi_medio
FROM gold.data_suicide.suicide_hdi_analysis;

-- 3.3 Verificar consistência temporal (sem gaps)
WITH anos_consecutivos AS (
    SELECT DISTINCT 
        country_code,
        year,
        LAG(year) OVER (PARTITION BY country_code ORDER BY year) AS ano_anterior
    FROM gold.data_suicide.suicide_hdi_analysis
)
SELECT 
    'Gaps Temporais' AS verificacao,
    COUNT(*) AS total_gaps,
    COUNT(DISTINCT country_code) AS paises_com_gaps
FROM anos_consecutivos
WHERE year - ano_anterior > 1;


In [0]:
-- 4.1 Insight 1: Relação HDI x Taxa de Suicídio
SELECT 
    'INSIGHT 1: HDI vs Suicídio' AS teste,
    hdi_category AS categoria_idh,
    ROUND(AVG(suicide_rate), 2) AS taxa_media_suicidio,
    COUNT(DISTINCT country_code) AS numero_paises
FROM gold.data_suicide.suicide_hdi_analysis
WHERE year >= 2015
GROUP BY hdi_category
ORDER BY taxa_media_suicidio DESC;

-- 4.2 Insight 3: Disparidade de Gênero
SELECT 
    'INSIGHT 3: Disparidade Gênero' AS teste,
    country_name AS pais,
    ROUND(gender_ratio, 2) AS razao_m_f,
    gap_category AS categoria
FROM gold.data_suicide.gender_disparity_analysis
WHERE year = (SELECT MAX(year) FROM gold.data_suicide.gender_disparity_analysis)
ORDER BY gender_ratio DESC
LIMIT 10;

-- 4.3 Insight 6: Alertas Críticos
SELECT 
    'INSIGHT 6: Alertas' AS teste,
    country_name AS pais,
    year AS ano,
    ROUND(suicide_rate, 2) AS taxa,
    alert_type AS tipo_alerta
FROM gold.data_suicide.significant_changes_alerts
WHERE year >= 2020
ORDER BY ABS(rate_change_pct) DESC
LIMIT 10;

-- 4.4 Insight Brasil 1: Evolução Brasil
SELECT 
    'BRASIL 1: Evolução' AS teste,
    year AS ano,
    ROUND(suicide_rate, 2) AS taxa_brasil,
    ROUND(regional_avg_rate, 2) AS media_americas,
    rank_in_region AS ranking_regional
FROM gold.data_suicide.country_rankings
WHERE country_code = 'BRA'
  AND year >= 2015
ORDER BY year DESC;

-- 4.5 Global Metrics - Dashboard
SELECT 
    'MÉTRICAS GLOBAIS' AS teste,
    year AS ano,
    ROUND(global_avg_rate, 2) AS taxa_media_global,
    countries_reporting AS paises_reportando
FROM gold.data_suicide.global_metrics
ORDER BY year DESC
LIMIT 5;


In [0]:
-- 5.1 Testar join Suicide + HDI
SELECT 
    'Join Suicide-HDI' AS validacao,
    COUNT(*) AS registros_com_join,
    COUNT(DISTINCT s.country_code) AS paises_matched
FROM gold.data_suicide.suicide_hdi_analysis s;

-- 5.2 Testar join com Gender Disparity
SELECT 
    'Join Gender Disparity' AS validacao,
    COUNT(*) AS total_registros,
    COUNT(DISTINCT country_code) AS paises_com_dados_genero
FROM gold.data_suicide.gender_disparity_analysis;


In [0]:
-- 6.1 KPIs principais para Dashboard 1
WITH dados_recentes AS (
    SELECT MAX(year) AS ano_max FROM gold.data_suicide.suicide_hdi_analysis
)
SELECT 
    -- KPI 1: Taxa Global Atual
    ROUND(AVG(suicide_rate), 2) AS taxa_global_atual,
    
    -- KPI 2: Total de países com dados
    COUNT(DISTINCT country_code) AS total_paises,
    
    -- KPI 3: Alertas críticos recentes
    (SELECT COUNT(DISTINCT country_code) 
     FROM gold.data_suicide.significant_changes_alerts 
     WHERE year >= (SELECT ano_max FROM dados_recentes) - 1
     AND alert_type LIKE '%Increase%') AS alertas_criticos,
    
    -- KPI 4: Ano mais recente
    MAX(year) AS ultimo_ano_disponivel
FROM gold.data_suicide.suicide_hdi_analysis
CROSS JOIN dados_recentes
WHERE year = ano_max;

In [0]:
-- 6.2 Dados para visualização principal (Evolução 2000-2024)
SELECT 
    year AS ano,
    ROUND(AVG(suicide_rate), 2) AS taxa_media_global,
    COUNT(DISTINCT country_code) AS paises_reportando
FROM gold.data_suicide.suicide_hdi_analysis
WHERE year >= 2000
GROUP BY year
ORDER BY year;